In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS #alternating least squares
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
data = spark.read.csv('movielens_ratings.csv', inferSchema = True, header = True)

In [4]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [5]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [6]:
training, test = data.randomSplit([0.8, 0.2])

In [7]:
als = ALS(maxIter = 5, regParam = 0.01, userCol = 'userId', itemCol = 'movieId', ratingCol= 'rating')

In [8]:
model = als.fit(training)

In [9]:
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      5|   2.0|    26| 0.03536099|
|      0|   1.0|    22|  1.2093825|
|      2|   3.0|     6|  0.5408547|
|      6|   2.0|    16| 0.18343863|
|      6|   1.0|    20|  2.4007208|
|      1|   1.0|     5|  0.6937717|
|      4|   1.0|     5|  2.1475542|
|      0|   1.0|    19| 0.62802815|
|      1|   4.0|    15|  0.7361165|
|      3|   1.0|    17|   0.904264|
|      1|   1.0|     4|  0.9637688|
|      0|   1.0|     8|  2.3572798|
|      6|   2.0|    23|  1.4485031|
|      4|   1.0|     7|  2.2730992|
|      2|   4.0|    10|  2.6977422|
|      7|   1.0|    10|  1.5891149|
|      1|   3.0|    25|-0.06983717|
|      6|   3.0|    24|   1.300911|
|      0|   1.0|    21|   2.249934|
|      4|   1.0|    14|  2.2523358|
+-------+------+------+-----------+
only showing top 20 rows



In [12]:
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'rating', predictionCol = 'prediction')

In [15]:
rmse = evaluator.evaluate(predictions)

In [16]:
print('RMSE')
print(rmse)

RMSE
1.7862506474921978


Not so good, but the data set is small.

In [25]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])

In [26]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     11|    11|
|     20|    11|
|     22|    11|
|     40|    11|
|     70|    11|
|     77|    11|
|     78|    11|
|     82|    11|
|     88|    11|
+-------+------+



In [28]:
recommendations = model.transform(single_user)

In [29]:
recommendations.orderBy('prediction', ascending = False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     77|    11|  2.7005477|
|     22|    11|  2.6771348|
|     11|    11|  2.4750543|
|     82|    11|  1.9378054|
|      9|    11|   1.620623|
|     40|    11|    1.10424|
|     20|    11| 0.32507002|
|     88|    11|-0.17970034|
|     78|    11| -0.4138257|
|     70|    11| -0.7658448|
+-------+------+-----------+

